<a href="https://colab.research.google.com/github/ahyz0569/Final-project-practice/blob/colab/200326_doc2vec_training_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Doc2vec 사용해보기

colab에서 doc2vec 모델을 학습시키고 문장 유사도를 확인해보자

In [0]:
# mecab 설치
# 출처: https://somjang.tistory.com/entry/Google-Colab에서-Mecab-koMecab-ko-dic-쉽게-사용하기 [솜씨좋은장씨]

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
! bash install_mecab-ko_on_colab190912.sh

In [0]:
!pwd

/content/Mecab-ko-for-Google-Colab


## Library import

In [0]:
import gensim
import sys
import pandas as pd
from konlpy.tag import Mecab
m = Mecab()
import re
import os
import numpy as np

# imports package
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import RegexpTokenizer

## 데이터 로드 및 전처리
- 특수 문자 제거(특수기호, 예/ ☆)

In [0]:
df=pd.read_csv('/content/drive/My Drive/Final-project/NLP/data/10000recipe_dataframe_for_token.csv',
                sep='\t',
                encoding='utf-8')
df.head()

,id,title,steps
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,먼저 전복을 물에 넣고는 삶아 줍니다. 전복을 삶은 중간에 오징어와 새우를 넣어서 ...
1,6894824,목우촌 캠핑요리 베이컨 야채말이,준비해둔 야채를 깨끗이 씻고 깻잎은 절반 대파는 한입크기 파프리카는 채 썰어줍니다 ...
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,단호박 찌는법 첫번째는 단호박은 반을 자르고 속안에 든 씨를 모두 제거해 줍니다. ...
3,6894822,풀무원 보글보글 된장찌게 끓이기,풀무원 정통된장을 냄비나 뚝배기에 물을 500ml넣고 끓여주세요.\\n(뒷면에 자세...
4,6894820,미숫가루 만들기,모든잡곡은 고두밥을 지어요 율무쌀은 가장 단단한 잡곡 이예요 불리지않고 밥하는것처럼...


In [0]:
## 특수문자 제거(기호들도 추가)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》☆★♡♥]', '', str(readData))
    return text

In [0]:
# title 컬럼 특수문자 제거하기
df['title']=df['title'].apply(cleanText)

In [0]:
df.head()

,id,title,steps
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,먼저 전복을 물에 넣고는 삶아 줍니다. 전복을 삶은 중간에 오징어와 새우를 넣어서 ...
1,6894824,목우촌 캠핑요리 베이컨 야채말이,준비해둔 야채를 깨끗이 씻고 깻잎은 절반 대파는 한입크기 파프리카는 채 썰어줍니다 ...
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,단호박 찌는법 첫번째는 단호박은 반을 자르고 속안에 든 씨를 모두 제거해 줍니다. ...
3,6894822,풀무원 보글보글 된장찌게 끓이기,풀무원 정통된장을 냄비나 뚝배기에 물을 500ml넣고 끓여주세요.\\n(뒷면에 자세...
4,6894820,미숫가루 만들기,모든잡곡은 고두밥을 지어요 율무쌀은 가장 단단한 잡곡 이예요 불리지않고 밥하는것처럼...


In [0]:
# 특수문자 제거 후 새로 저장
df.to_csv('/content/drive/My Drive/Final-project/NLP/data/10000recipe_dataframe_for_token.csv',
            sep='\t',
            encoding='utf-8',
            index=False)

## 레시피 제목 텍스트 전처리 하기

- 형태소 분석을 해서 명사와 형용사만 사용

In [0]:
# 데이터 프레임 불러오기
df=pd.read_csv('/content/drive/My Drive/Final-project/NLP/data/10000recipe_dataframe_for_token.csv',
                sep='\t',
                encoding='utf-8')
df.head()

,id,title,steps
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,먼저 전복을 물에 넣고는 삶아 줍니다. 전복을 삶은 중간에 오징어와 새우를 넣어서 ...
1,6894824,목우촌 캠핑요리 베이컨 야채말이,준비해둔 야채를 깨끗이 씻고 깻잎은 절반 대파는 한입크기 파프리카는 채 썰어줍니다 ...
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,단호박 찌는법 첫번째는 단호박은 반을 자르고 속안에 든 씨를 모두 제거해 줍니다. ...
3,6894822,풀무원 보글보글 된장찌게 끓이기,풀무원 정통된장을 냄비나 뚝배기에 물을 500ml넣고 끓여주세요.\\n(뒷면에 자세...
4,6894820,미숫가루 만들기,모든잡곡은 고두밥을 지어요 율무쌀은 가장 단단한 잡곡 이예요 불리지않고 밥하는것처럼...


In [0]:
df.shape

(54738, 3)

In [0]:
# 레시피 제목과 id만 추려서 데이터프레임 새로 만들기
title_df = df[['id', 'title']]
title_df.head()

NameError: ignored

In [0]:
# 문장을 토큰화 시킬 함수 선언
def tokenizer_Mecab_morphs(doc):
    return m.morphs(str(doc))

def tokenizer_Mecab_noun(doc):
    return m.nouns(str(doc))

def tokenizer_Mecab_pos(doc):
    return m.pos(str(doc), flatten=True)

In [0]:
# 함수 사용 테스트
print(tokenizer_Mecab_morphs('더 좋은 한 끼 곤약젤리'))
print(tokenizer_Mecab_noun('더 좋은 한 끼 곤약젤리'))
print(tokenizer_Mecab_pos('더 좋은 한 끼 곤약젤리'))

['더', '좋', '은', '한', '끼', '곤약', '젤리']
['끼', '곤약', '젤리']
[('더', 'MAG'), ('좋', 'VA'), ('은', 'ETM'), ('한', 'MM'), ('끼', 'NNBC'), ('곤약', 'NNG'), ('젤리', 'NNG')]


In [0]:
# 명사와 형용사만 찾을 정규 표현식 생성
find_pos = re.compile(r'^N.*|VA|^M.*|XR(?!VV+)')

In [0]:
# 문장을 토큰화 한 후 명사와 형용사만 추가하는 함수 선언
def preprocessing_text(doc):
    doc = str(doc)
    token_list=[]
    tokens = m.pos(doc, flatten=True)
    for word, pos in tokens:
        if find_pos.match(pos):
            token_list.append(word)
    return token_list

In [0]:
# title 컬럼을 토큰화시켜서 새로운 컬럼('token_title') 생성
title_df['token_title']=title_df['title'].apply(preprocessing_text)

In [0]:
title_df.head()

,id,title,token_title
0,6894825,냉장고를부탁해 이연복 전복영양밥만들기 전복과 오징어새우가 들어간 볶음밥에 모닝글로리...,"[냉장고, 부탁, 이연복, 전복, 영양, 밥, 전복, 오징어, 새우, 볶음밥, 모닝..."
1,6894824,목우촌 캠핑요리 베이컨 야채말이,"[목, 우촌, 캠핑, 요리, 베이컨, 야채, 말]"
2,6894823,단호박라떼 전자렌지로 단호박 찌는법,"[단호박, 라떼, 전자, 렌지, 단호박, 법]"
3,6894822,풀무원 보글보글 된장찌게 끓이기,"[풀무원, 보글보글, 된장]"
4,6894820,미숫가루 만들기,[미숫가루]


## Doc2vec 모델 학습시키기

In [0]:
# Doc2vec 입력 형식에 맞게 데이터 수정
# tags = 문서 ID
# words = 단어 list
title_doc_df = title_df[['id', 'token_title']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in title_doc_df]

In [0]:
# Training
max_epochs=10

model=Doc2Vec(window=10,
             vector_size=150,
             alpha=0.025,
             min_alpha=0.025,
             min_count=2,
             dm = 1,
             negative = 5,
             seed=9999)

model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
               total_examples=model.corpus_count,
               epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    model.min_alpha = model.alpha

# history = model.fit_generator(train_generator, epochs=50, validation_data = val_generator,
#                               callbacks=[checkpoint, early_stopping])

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['loss', 'val_loss', 'acc', 'val_acc'], loc='upper left')
# plt.show()

# 모델 저장하기
from gensim.test.utils import get_tmpfile

# temp_path = get_tmpfile('title_doc2vec_0327.pkl')
DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'
model.save(os.path.join(DOC2VEC_DRIVE_PATH, "title_doc2vec_0327.pkl"))

model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


AttributeError: ignored

In [0]:
# 모델 저장하기
from gensim.test.utils import get_tmpfile

# temp_path = get_tmpfile('title_doc2vec_0327.pkl')
DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'
model.save(os.path.join(DOC2VEC_DRIVE_PATH, "title_doc2vec_0327.pkl"))

model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.total_train_time

201.17374503699898

False

In [0]:
# # Training
# max_epochs=10

# model=Doc2Vec(window=10,
#              vector_size=150,
#              alpha=0.025,
#              min_alpha=0.025,
#              min_count=2,
#              dm = 1,
#              negative = 5,
#              seed=9999)

# model.build_vocab(tagged_data)
# for epoch in range(max_epochs):
#     print('iteration {0}'.format(epoch))
#     model.train(tagged_data,
#                total_examples=model.corpus_count,
#                epochs=model.iter)
#     # decrease the learning rate
#     model.alpha -= 0.002
#     model.min_alpha = model.alpha

In [0]:
# # 모델 저장하기
# from gensim.test.utils import get_tmpfile

# temp_path = get_tmpfile('title_doc2vec_0325.pkl')
# DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'
# model.save(os.path.join(DOC2VEC_DRIVE_PATH, "title_doc2vec_0327.pkl"))

In [0]:
# # temp 파일 삭제
# model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [0]:
# DOC2VEC_DRIVE_PATH = '/content/drive/My Drive/Final-project/NLP/model/'

## 문장 유사도 확인하기

In [0]:
# model load
DOC2VEC_MODEL_PATH = os.path.join(DOC2VEC_DRIVE_PATH, "title_doc2vec_0327.pkl")
model = Doc2Vec.load(DOC2VEC_MODEL_PATH)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
print(gensim)

In [0]:
!pip install --upgrade h5py
!pip install --upgrade numpy

Requirement already up-to-date: h5py in /usr/local/lib/python3.6/dist-packages (2.10.0)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.18.2)


In [0]:
# 새로운 문장을 넣어 유사도 확인하기
model.random.seed(8000)

example_list = preprocessing_text('참치')
inferred_vector = model.infer_vector(example_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=20)
for rd in return_docs:
    for des in title_df[title_df['id'] == rd[0]]['title']:
        print(rd[0], rd[1], des)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


6846767 0.7780711054801941 생생정보 참치김치볶음밥 황금레시피
6859727 0.7554196715354919 참치와 닭가슴살이 들어간 길거리 토스트
6906711 0.7363095283508301 참치사골미역국
6899085 0.7068617939949036 두부조림 칼칼한 두부참치 두루치기
6854477 0.6915662288665771 참치김치볶음밥 만들기 예쁜 곰돌이 모양으로
6879593 0.6904956102371216 참치스팸햄샌드위치만들기 스팸햄과 참치가 들어간 간식 참치스팸햄샌드위치
6706014 0.6856033205986023 김밥 예쁘게 싸는 방법 참치김밥 삼겹살김밥
6896897 0.6570698618888855 국물이 일품인 참치 오징어찌개
6924060 0.6378894448280334 간단밑반찬 참치총각김치볶음
3435391 0.6363502740859985 참치달걀마요 피자토스트
6914804 0.633748471736908 제철 음식 참다랑어로 참치 후토마끼 만들기
6856180 0.631584107875824 장아찌 드레싱누들고소한 참치에 장아찌로 입맛을 잡아요
6864263 0.6207029223442078 참치주먹밥베이컨떡말이
6910085 0.6202589273452759 참치라면 레시피 숙주넣어 아삭하게
6913396 0.6198413968086243 간편식상품화 크림 데리 참치파스타
6876083 0.615755558013916 사과가들어가 달콤상콤한 참치또띠아롤
6921856 0.6076948642730713 부들부들 맛나는 참치두부두루치기
6288437 0.6000051498413086 참치 카레라이스
6913611 0.5963339805603027 아이들매일반찬 참치김치찌개
6921474 0.5903058648109436 마파두부 돼지고기 대신 참치로 만든 마파두부


In [0]:
model.random.seed(8000)

example_list = preprocessing_text('단짠단짠의 대패덮밥')
inferred_vector = model.infer_vector(example_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=20)
for rd in return_docs:
    for des in title_df[title_df['id'] == rd[0]]['title']:
        print(rd[0], rd[1], des)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


6915535 0.7545669078826904 달큰한 양파덮밥
6865517 0.7505785822868347 돈까스 덮밥
6845640 0.748032808303833 치킨너겟덮밥 만들기
6920864 0.7455601692199707 쉬림프 덮밥 만들기
6916795 0.7446784377098083 암예방레시피마늘쫑 덮밥
6894131 0.7403937578201294 스크램블 에그 덮밥
6919390 0.740318238735199 칠리새우덮밥 만들기 새우요리
6888174 0.7389644980430603 덮밥처럼 먹는 햄버그스테이크하와이식 함박스테이크
6854040 0.7384890913963318 치키마요덮밥
6912867 0.7337858080863953 가정간편식 삼치구이 덮밥
6867860 0.732214629650116 간장 돼지 불고기 덮밥
6890199 0.7296136617660522 소라장 덮밥
6842340 0.7264766693115234 먹다 남은 치킨 활용법 치킨마요덮밥
6913339 0.7259777784347534 가정간편식새우마요덮밥
6879245 0.724875807762146 새우장 덮밥
6868557 0.7173271775245667 오징어덮밥 만들기
6883216 0.7170000076293945 닭가슴살 덮밥으로 즐겨요
6846693 0.7141279578208923 오징어 튀김 덮밥 만들기
2690813 0.7133302688598633 데리야키 덮밥
6899178 0.7102718353271484 오징어다리 볶아 덮밥으로 먹었습니다 오징어덮밥 오징어볶음


In [0]:
model.random.seed(8000)

test_txt = cleanText('아삭아삭하고 시원한 오이소박이')
example_list = preprocessing_text(test_txt)
inferred_vector = model.infer_vector(example_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=20)
for rd in return_docs:
    for des in title_df[title_df['id'] == rd[0]]['title']:
        print(rd[0], rd[1], des)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


6912730 0.7404549717903137 시원한 얼갈이 된장국
4507979 0.7396938800811768 구수하고 시원한 배추된장국만들기
6881497 0.731243371963501 시원한 배추 달래 된장국
6891900 0.7193628549575806 열무김치 담그기 시원한 얼갈이 열무김치 담그는법
6923058 0.7172775268554688 시원한 홍합미역국 끓이기
6851748 0.7104392051696777 아삭하고 시원하게오이소박이 만들기
6917343 0.7088552713394165 시원한 노각냉국
6855524 0.6968193650245667 아삭하고 시원하게 깍두기만들기
5593183 0.6887080073356628 시원한 감자국
6860008 0.688239336013794 시원한 밀크티
6892578 0.6866437196731567 시원하게 호로록 메밀소바
6903661 0.6788150668144226 시원한 어묵탕 바른 어묵으로 끓인 어묵탕
6901035 0.6738768219947815 시원하고 뽀얀 황태국
4469459 0.6699370741844177 얼큰하고 시원한 동태찌개 긇이기
6855419 0.6695454120635986 시원한 냉메밀국수
6837990 0.6682031750679016 동태찌개 시원하게 끓이기
6853944 0.6622316837310791 시원한 오이미역냉국
6874886 0.6619391441345215 시원한 무우국
6891518 0.6617140173912048 여름철별미 시원한 미역오이냉국 만들기
6873115 0.6602320671081543 숙취해소에도 좋은 시원한 재첩국 만들기


In [0]:
# 단어로 유사도 확인해보기
model.most_similar('덮밥',topn=20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('마파두부', 0.47617700695991516),
 ('김치볶음밥', 0.4589522182941437),
 ('볶음밥', 0.4451290965080261),
 ('밥', 0.413921982049942),
 ('볶음', 0.41321995854377747),
 ('돈부리', 0.4131929874420166),
 ('잡채', 0.4063357710838318),
 ('두부조림', 0.4055812358856201),
 ('동', 0.3950354754924774),
 ('짜장', 0.3765466511249542),
 ('무침', 0.3756636083126068),
 ('계란덮밥', 0.3751850128173828),
 ('두부덮밥', 0.3706231415271759),
 ('달걀덮밥', 0.363157719373703),
 ('조림', 0.36021414399147034),
 ('오므라이스', 0.3601547181606293),
 ('불고기', 0.3594944179058075),
 ('샐러드', 0.3579161465167999),
 ('콩나물밥', 0.35724809765815735),
 ('소스', 0.35698169469833374)]

## 결과

- 단어를 넣어 유사도를 확인한 경우에는 유사 단어가 잘 나오지만 문장을 넣었을 경우에는 원하는 결과 값이 잘 나오지 않는 것을 확인하였음
- 문장을 토큰화 한 후 불용어 제거를 하지 않고 모델에 학습시켰더니 결과가 좋지 않은 것으로 보인다
- 불용어 제거를 한 후 모델의 성능을 확인 해봐야 겠다.